In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import os
path = "C:\\Users\\Elite\\Downloads\\chromedriver_win32\\chromedriver"

#### 링크 가져오기 ####
def get_link(category):

    driver = webdriver.Chrome(path)
    driver.implicitly_wait(10)
    driver.get("https://www.iherb.com/search?kw="+category+"&sr=2")
    
    product_link = []
    
    for i in range(2):
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        http = soup.select("div.product-inner.product-inner-wide > a.product-image")
        for i in http:
            product_link.append(i.get('content'))
        
    
    product_link = product_link[0:30]
        

        
    #링크 만드는데 필요한 상품명 모음
    name = []
    for i in range(30):
        name.append(product_link[i].split('/')[4])
    
    # 상품코드숫자만 모음
    code = []
    for n in range(30):
        code.append(product_link[n].split('/')[5])

    # 상품명과 상품코드숫자를 합쳐서 링크를 만듬 (pr은 제품클림, r로 바꾸면 리뷰창으로 바로 들어감)
    total_link = [] # 링크 모음
    for k in range(30):
        total_link.append('https://www.iherb.com/r/' + name[k] +"/"+ code[k])
    return total_link

    driver.close()
    
    
#### 상품 명 가져오기 ####    
def get_product_name(category):
    driver = webdriver.Chrome(path)
    driver.implicitly_wait(10)
    driver.get("https://www.iherb.com/search?kw="+category+"&sr=2")
    
    product_name = [] #실제 상품명
    for i in range(2):
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        http = soup.select("div.product-inner.product-inner-wide > a.product-image")
    
        for y in http:
            product_name.append(y.get('title'))
    
    product_name = product_name[0:30]
    return product_name
    

#### 딕셔너리를 엑셀로 변환 ####

def to_excel(categoty):
    writer = pd.ExcelWriter('iherb_T_'+category+'.xlsx')
    df_iherb.to_excel(writer,'sheet1')
    writer.save()
    

In [2]:
#### 카테고리 ####
category_all = ['Omega 3', 'vitamin b', 'vitamin c', 'vitamin d', 'Magnesium', 'calcium', 'Amino Acids', 'iron', 'collagen']

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from pandas import Series, DataFrame
import os
path = "C:\\Users\\Elite\\Downloads\\chromedriver_win32\\chromedriver"

for category in category_all:
    print(category)
    total_url = get_link(category) 
    title= get_product_name(category) 
    
    comments_all = []
    for i in range(30):
        driver = webdriver.Chrome(path)
        driver.implicitly_wait(10)
        print(total_url[i])
        driver.get(total_url[i])
        reviews_list = []
        pages = driver.find_element_by_css_selector('div.paging-counts').text
        pages = pages[8:-6]
        print(pages)
        if pages != '' and int(pages) >= 10:
            pages = int(pages)//10
        else:
            pages = 1
    
        pages = min(pages, 10)
        #driver.close()
    
        for n in range(pages):
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            reviews = soup.select("div.review-core-section > div.review-text")
        
            reviews_list.extend([v.get_text() for v in reviews])
        
            button = driver.find_element_by_css_selector("div.paging > button.arrow-button > i.icon-recentlyviewedarrowright")
            driver.execute_script("arguments[0].click();",button)
            time.sleep(3)
            #driver.close()
        comments_all.append(reviews_list)
    
   
    dict_iherb = {}
    for i in range(30):
        dict_iherb[title[i]] = comments_all[i]
       
    
    df_iherb = pd.DataFrame.from_dict(dict_iherb, orient = 'index')
    
    # 엑셀로 변환 함수
    to_excel(category_all)

Omega 3
https://www.iherb.com/r/California-Gold-Nutrition-Omega-3-Premium-Fish-Oil-100-Fish-Gelatin-Softgels/62118
1206
https://www.iherb.com/r/Now-Foods-Omega-3-180-EPA-120-DHA-200-Softgels/323
1839
https://www.iherb.com/r/California-Gold-Nutrition-Omega-3-Premium-Fish-Oil-240-Fish-Gelatin-Softgels/86598
1206
https://www.iherb.com/r/Natrol-Omega-3-Fish-Oil-Natural-Lemon-Flavor-1-000-mg-150-Softgels/2242
437
https://www.iherb.com/r/California-Gold-Nutrition-Omega-800-by-Madre-Labs-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-Triglyceride-Form-1000-mg-30-Fish-Gelatin-Softgels/82845
233
https://www.iherb.com/r/Now-Foods-Ultra-Omega-3-500-EPA-250-DHA-90-Softgels/76073
354
https://www.iherb.com/r/California-Gold-Nutrition-Omega-800-by-Madre-Labs-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-Triglyceride-Form-1000-mg-90-Fish-Gelatin-Softgels/85180
233
https://www.iherb.com/r/Solgar-Omega-3-Fish-Oil-Concentrate-240-Softgels/23555
166
https://www.iherb.com/r/Sports-Research-Antarctic-Krill-Oil-with-As

409
https://www.iherb.com/r/California-Gold-Nutrition-Collagen-UP-Marine-Sourced-Collagen-Peptides-Hyaluronic-Acid-Vitamin-C-Unflavored-7-26-oz-206-g/64903
645
https://www.iherb.com/r/ChildLife-Essentials-Liquid-Vitamin-C-Natural-Orange-Flavor-4-fl-oz-118-5-mL/6356
406
https://www.iherb.com/r/California-Gold-Nutrition-Gold-C-Vitamin-C-Ascorbic-Acid-1-000-mg-60-Veggie-Capsules/61864
707
https://www.iherb.com/r/California-Gold-Nutrition-Gold-C-Vitamin-C-L-Ascorbic-Acid-1-000-mg-240-Veggie-Capsules/61865
707
vitamin d
https://www.iherb.com/r/Now-Foods-Vitamin-D-3-5-000-IU-240-Softgels/22335
1183
https://www.iherb.com/r/California-Gold-Nutrition-Baby-Vitamin-D3-Drops-10-mcg-400-IU-34-fl-oz-10-ml/65958
28
https://www.iherb.com/r/ChildLife-Vitamin-D3-Natural-Berry-Flavor-1-fl-oz-29-6-ml/24742
118
https://www.iherb.com/r/21st-Century-D3-25-mcg-1000-IU-110-Tablets/15030
713
https://www.iherb.com/r/Healthy-Origins-Vitamin-D3-10-000-IU-360-Softgels/21298
882
https://www.iherb.com/r/Now-Foods-Vit

60
https://www.iherb.com/r/Solgar-Calcium-Magnesium-Plus-Zinc-250-Tablets/8693
103
https://www.iherb.com/r/21st-Century-Calcium-Magnesium-Zinc-D3-90-Tablets/10695
336
https://www.iherb.com/r/ChildLife-Liquid-Calcium-with-Magnesium-Natural-Orange-Flavor-16-fl-oz-474-ml/9434

https://www.iherb.com/r/Now-Foods-Calcium-Magnesium-250-Tablets/453
140
https://www.iherb.com/r/Nature-s-Bounty-Calcium-Magnesium-Zinc-with-Vitamin-D3-100-Coated-Caplets/32229
116
https://www.iherb.com/r/Nature-s-Plus-Source-of-Life-Animal-Parade-Calcium-Children-s-Chewable-Supplement-Natural-Vanilla-Sundae-Flavor-90-Animal-Shaped-Tablets/7531
130
https://www.iherb.com/r/GummiKing-Calcium-Plus-Vitamin-D-for-Kids-60-Gummies/34010
114
Amino Acids
https://www.iherb.com/r/Now-Foods-L-Cysteine-500-mg-100-Tablets/538
251
https://www.iherb.com/r/Now-Foods-L-Arginine-1-000-mg-120-Tablets/411
196
https://www.iherb.com/r/Now-Foods-L-Tyrosine-500-mg-120-Capsules/836
329
https://www.iherb.com/r/California-Gold-Nutrition-L-Gluta

KeyboardInterrupt: 